# Cleaning Data

In this section
* grab data from the csv that previously scrape from Seek
* clean duplicate data
* classify the location to be [ACT, NSW, NT, QLD, SA, TAS, VIC, WA]
* create a new column to specify the salary range

There are six states in Australia: New South Wales, Queensland, South Australia, Tasmania, Victoria and Western Australia.
There are two Territories

In [1]:
# import necessary packages
import time
import datetime as dt
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import style
import seaborn as sns

style.use('ggplot')

%matplotlib inline

## Load data from CSV

In [2]:
# import the data from csv

In [3]:
sixty = pd.read_csv('./datasets/dataSixty.csv')
eighty = pd.read_csv('./datasets/dataEighty.csv')
hundred = pd.read_csv('./datasets/dataHundred.csv')
onetwenty = pd.read_csv('./datasets/dataOnetwenty.csv')
onefifty = pd.read_csv('./datasets/dataOnefifty.csv')
twohundred = pd.read_csv('./datasets/dataTwohundred.csv')
twofifty = pd.read_csv('./datasets/dataTwofifty.csv')

In [4]:
sixty.head(2)

,Unnamed: 0,title,link,salary,location,company,classification,summary
0,0,Category Data Assistant,/job/37426653?type=promoted,NaN,Melbourne,TerryWhite Chemmart,Administration & Office Support,About Us TerryWhite Chemmart (TWCM) is Austral...
1,1,Data Administrator,/job/37381361?type=promoted,NaN,Sydney,Krunchbox Operations,Information & Communication Technology,About the business and the role This could be...


In [5]:
# all of them have ['Unnamed: 0'] columns, drop them all
sixty.drop(columns = ['Unnamed: 0'], inplace=True)
eighty.drop(columns = ['Unnamed: 0'], inplace=True)
hundred.drop(columns = ['Unnamed: 0'], inplace=True)
onetwenty.drop(columns = ['Unnamed: 0'], inplace=True)
onefifty.drop(columns = ['Unnamed: 0'], inplace=True)
twohundred.drop(columns = ['Unnamed: 0'], inplace=True)
twofifty.drop(columns = ['Unnamed: 0'], inplace=True)

## Examine data for data Cleaning 

In [6]:
sixty.head()

,title,link,salary,location,company,classification,summary
0,Category Data Assistant,/job/37426653?type=promoted,NaN,Melbourne,TerryWhite Chemmart,Administration & Office Support,About Us TerryWhite Chemmart (TWCM) is Austral...
1,Data Administrator,/job/37381361?type=promoted,NaN,Sydney,Krunchbox Operations,Information & Communication Technology,About the business and the role This could be...
2,Data Entry,/job/37561354?type=standard,NaN,Perth,Page Personnel Office Support,Administration & Office Support,Provide Data Entry support for a 4 week period...
3,Accounts Assistant / Data Entry,/job/37560786?type=standard,NaN,Perth,Perth Rad Clinic Limited,Administration & Office Support,About Perth Radiological Clinic Perth Radiolog...
4,Admin /Data Entry,/job/37560693?type=standard,Up to $27.23 p.h. + Super,"Wollongong, Illawarra & South Coast",Veritas Recruitment,Administration & Office Support,The Company This market leading company requi...


In [7]:
# checking null colunmns - found mainly salary and company.... 
# null we know roughly the bracket we put them in...... so I can fill it up with some median later
# company are not really relevant...no need cleaning
print ('sixty\n', sixty.isnull().sum())
print ('------------------------------------')
print ('eighty\n', eighty.isnull().sum())
print ('------------------------------------')
print ('hundred\n', hundred.isnull().sum())
print ('------------------------------------')
print ('onetwenty\n', onetwenty.isnull().sum())
print ('------------------------------------')
print ('onefifty\n', onefifty.isnull().sum())
print ('------------------------------------')
print ('twohundred\n', twohundred.isnull().sum())
print ('------------------------------------')
print ('twofifty\n', twofifty.isnull().sum())

sixty
 title                0
link                 0
salary            2922
location             0
company            214
classification       0
summary              0
dtype: int64
------------------------------------
eighty
 title                0
link                 0
salary            2728
location             0
company            139
classification       0
summary              0
dtype: int64
------------------------------------
hundred
 title                0
link                 0
salary            2703
location             0
company             87
classification       0
summary              2
dtype: int64
------------------------------------
onetwenty
 title                0
link                 0
salary            2725
location             0
company             62
classification       0
summary              0
dtype: int64
------------------------------------
onefifty
 title                0
link                 0
salary            2458
location             0
company            

In [8]:
# Since in the Sixty has the highest number, we will use those locations information to classified them
print ('sixty ', sixty.location.nunique())
print ('eighty ', eighty.location.nunique())
print ('hundred ', hundred.location.nunique())
print ('onetwenty ', onetwenty.location.nunique())
print ('onefifty ', onefifty.location.nunique())
print ('twohundred ', twohundred.location.nunique())
print ('twofifty ', twofifty.location.nunique())

sixty  63
eighty  58
hundred  57
onetwenty  59
onefifty  47
twohundred  31
twofifty  25


In [9]:
# max classificatin is 29 and are already unique... so nothing to do there
print ('sixty ', sixty.classification.nunique())
print ('eighty ', eighty.classification.nunique())
print ('hundred ', hundred.classification.nunique())
print ('onetwenty ', onetwenty.classification.nunique())
print ('onefifty ', onefifty.classification.nunique())
print ('twohundred ', twohundred.classification.nunique())
print ('twofifty ', twofifty.classification.nunique())
sixty.classification.unique()

sixty  29
eighty  28
hundred  29
onetwenty  29
onefifty  29
twohundred  28
twofifty  24


array(['Administration & Office Support',
       'Information & Communication Technology',
       'Manufacturing, Transport & Logistics', 'Accounting',
       'Healthcare & Medical', 'Advertising, Arts & Media',
       'Trades & Services', 'Retail & Consumer Products',
       'Education & Training', 'Insurance & Superannuation',
       'Mining, Resources & Energy', 'Engineering',
       'Government & Defence', 'Sales', 'Construction',
       'Science & Technology', 'Banking & Financial Services',
       'Call Centre & Customer Service', 'Marketing & Communications',
       'Community Services & Development',
       'Farming, Animals & Conservation', 'Consulting & Strategy',
       'Human Resources & Recruitment', 'Real Estate & Property', 'Legal',
       'Design & Architecture', 'Hospitality & Tourism',
       'CEO & General Management', 'Sport & Recreation'], dtype=object)

1. Salary are in different format and contains null values but we know which brackets they are in
2. Notice that the Location are not based on States or Territories, we will need to convert them
3. Summary - Some advertisement posted multiple times

###  Data Manipulation and Cleaning
    1. Drop duplicates based on summary

In [10]:
# shows number of duplicates
sixty[sixty.duplicated(['summary'], keep='first')].shape

(406, 7)

In [11]:
# drop all duplicates except first one and show shapes
sixty.drop_duplicates(subset=['summary'], inplace=True)
sixty.shape

(3614, 7)

In [12]:
# shows number of duplicates
eighty[eighty.duplicated(['summary'], keep='first')].shape

(516, 7)

In [13]:
# drop all duplicates except first one and show shapes
eighty.drop_duplicates(subset=['summary'], inplace=True)
eighty.shape

(3540, 7)

In [14]:
# shows number of duplicates
hundred[hundred.duplicated(['summary'], keep='first')].shape

(703, 7)

In [15]:
# drop all duplicates except first one and show shapes
hundred.drop_duplicates(subset=['summary'], inplace=True)
hundred.shape

(3333, 7)

In [16]:
# shows number of duplicates
onetwenty[onetwenty.duplicated(['summary'], keep='first')].shape

(684, 7)

In [17]:
# drop all duplicates except first one and show shapes
onetwenty.drop_duplicates(subset=['summary'], inplace=True)
onetwenty.shape

(3346, 7)

In [18]:
# shows number of duplicates
onefifty[onefifty.duplicated(['summary'], keep='first')].shape

(671, 7)

In [19]:
# drop all duplicates except first one and show shapes
onefifty.drop_duplicates(subset=['summary'], inplace=True)
onefifty.shape

(3041, 7)

In [20]:
# shows number of duplicates
twohundred[twohundred.duplicated(['summary'], keep='first')].shape

(448, 7)

In [21]:
# drop all duplicates except first one and show shapes
twohundred.drop_duplicates(subset=['summary'], inplace=True)
twohundred.shape

(2228, 7)

In [22]:
# shows number of duplicates
twofifty[twofifty.duplicated(['summary'], keep='first')].shape

(149, 7)

In [23]:
# drop all duplicates except first one and show shapes
twofifty.drop_duplicates(subset=['summary'], inplace=True)
twofifty.shape

(1055, 7)

###  Data Manipulation and Cleaning
    2. Modify the locations to make it into the correct States and Territories
    
    Steps
    * Using Sixty to show 63 different locations list and put them into the correct grouping
    * Create function to use
    * Apply it to each DF and Save it to new Columns
    # Remove all those that are overseas ads

In [24]:
# find unique locations, classified them in to the correct States
location_list = sixty.location.unique()
location_list

array(['Melbourne', 'Sydney', 'Perth',
       'Wollongong, Illawarra & South Coast', 'Brisbane', 'Gold Coast',
       'Lismore & Far North Coast', 'Western QLD',
       'Yarra Valley & High Country', 'Hobart',
       'Tamworth & North West NSW', 'Gladstone & Central QLD', 'Adelaide',
       'South West Coast VIC', 'ACT', 'Sunshine Coast', 'Albury Area',
       'Newcastle, Maitland & Hunter', 'Dubbo & Central NSW',
       'Launceston & North East', 'Cairns & Far North', 'Darwin',
       'Riverland & Murray Mallee', 'Mackay & Coalfields',
       'Bairnsdale & Gippsland', 'Ballarat & Central Highlands',
       'Rockhampton & Capricorn Coast', 'Northern QLD',
       'Bunbury & South West', 'Port Macquarie & Mid North Coast',
       'Coffs Harbour & North Coast', 'Tumut, Snowy & Monaro',
       'Coober Pedy & Outback SA', 'Hervey Bay & Fraser Coast',
       'Mildura & Murray', 'Wagga Wagga & Riverina',
       'Toowoomba & Darling Downs', 'Katherine & Northern Australia',
       'Southern Hi

In [25]:
ACT = ['ACT']

NSW =['NSW', 'Sydney', 'Albury Area', 'Blue Mountains & Central West', 'Coffs Harbour & North Coast', 
      'Gosford & Central Coast', 'Lismore & Far North Coast', 'Newcastle, Maitland & Hunter',
       'Port Macquarie & Mid North Coast', 'Richmond & Hawkesbury', 'Southern Highlands & Tablelands',
       'Tumut, Snowy & Monaro', 'Wagga Wagga & Riverina','Wollongong, Illawarra & South Coast']

NT = ['NT', 'Darwin', 'Alice Springs & Central Australia']

QLD = ['QLD', 'Brisbane', 'Bundaberg & Wide Bay Burnett', 'Cairns & Far North', 'Gold Coast', 
       'Hervey Bay & Fraser Coast', 'Katherine & Northern Australia', 'Mackay & Coalfields', 
       'Rockhampton & Capricorn Coast', 'Somerset & Lockyer', 'Sunshine Coast', 'Toowoomba & Darling Downs']

SA = ['SA', 'Adelaide', 'Adelaide Hills & Barossa', 'Fleurieu Peninsula & Kangaroo Island',
      'Mt Gambier & Limestone Coast', 'Riverland & Murray Mallee', 
      'Whyalla & Eyre Peninsula', 'Yorke Peninsula & Clare Valley']

TAS = ['TAS', 'Hobart', 'Devonport & North West', 'Launceston & North East']

VIC = ['VIC', 'Melbourne', 'Bairnsdale & Gippsland', 'Ballarat & Central Highlands',
       'Bendigo, Goldfields & Macedon Ranges', 'Horsham & Grampians','Mornington Peninsula & Bass Coast',
       'Shepparton & Goulburn Valley', 'West Gippsland & Latrobe Valley', 'Yarra Valley & High Country']

WA = ['WA', 'Perth', 'Albany & Great Southern', 'Bairnsdale & Gippsland', 'Bendigo, Goldfields & Macedon Ranges', 
      'Broome & Kimberley', 'Bunbury & South West', 'Kalgoorlie, Goldfields & Esperance','Geraldton, Gascoyne & Midwest', 
      'Mandurah & Peel', 'Mildura & Murray', 'Northam & Wheatbelt', 'Port Hedland, Karratha & Pilbara']

In [26]:
def get_location_states(location):
    
    if len(location) <= 3:
        #Already in the correct States or Territories
        return location
    
    #check if the data has the states inside, get the last 3 letters and remove space
    loc = location[-3:].strip()
    
    if loc.isupper():
        # return the states if all 3 or 2 letters are upper case
        return loc
    
    if location in NSW:
        return 'NSW'
    
    if location in VIC:
        return 'VIC'
    
    if location in QLD:
        return 'QLD'
    
    if location in WA:
        return 'WA'
    
    if location in SA:
        return 'SA'
    
    if location in TAS:
        return 'TAS'
    
    if location in NT:
        return 'NT'
 
    print (location)
    return 'NA'


In [27]:
sixty['state'] = sixty['location'].apply(get_location_states)
print(sixty['state'].unique())
sixty.head(2)

['VIC' 'NSW' 'WA' 'QLD' 'TAS' 'SA' 'ACT' 'NT']


,title,link,salary,location,company,classification,summary,state
0,Category Data Assistant,/job/37426653?type=promoted,NaN,Melbourne,TerryWhite Chemmart,Administration & Office Support,About Us TerryWhite Chemmart (TWCM) is Austral...,VIC
1,Data Administrator,/job/37381361?type=promoted,NaN,Sydney,Krunchbox Operations,Information & Communication Technology,About the business and the role This could be...,NSW


In [28]:
eighty['state'] = eighty['location'].apply(get_location_states)
print(eighty['state'].unique())
eighty.head(2)

Ireland
['NSW' 'VIC' 'QLD' 'WA' 'ACT' 'SA' 'TAS' 'NT' 'NA']


,title,link,salary,location,company,classification,summary,state
0,Electrician / Data Cabler,/job/37555358?type=promoted,"$60,000 - $79,999",Sydney,AMR Security,Trades & Services,About the roleWe are looking for an Electricia...,NSW
1,Shift Technician - Electrical (Data Centre),/job/37326042?type=promoted,NaN,Sydney,BGIS,Trades & Services,"BGIS, a Brookfield Company, is a leading provi...",NSW


In [29]:
hundred['state'] = hundred['location'].apply(get_location_states)
print(hundred['state'].unique())
hundred.head(2)

Ireland
['NSW' 'VIC' 'WA' 'QLD' 'ACT' 'NT' 'SA' 'TAS' 'NA']


,title,link,salary,location,company,classification,summary,state
0,Shift Technician - Electrical (Data Centre),/job/37326042?type=promoted,NaN,Sydney,BGIS,Trades & Services,"BGIS, a Brookfield Company, is a leading provi...",NSW
1,Enterprise Data & Reporting Lead,/job/37393509?type=promoted,Base + Super + Profit Share,Melbourne,SEEK Limited,Information & Communication Technology,About SEEK SEEK is the global leader in the cr...,VIC


In [30]:
hundred[hundred['state'] == 'NA']

,title,link,salary,location,company,classification,summary,state
2128,Design Engineer,/job/37426942?type=promoted,NaN,Ireland,PM Group,Engineering,We are currently looking to recruit an experie...,NA


In [31]:
onetwenty['state'] = onetwenty['location'].apply(get_location_states)
print(onetwenty['state'].unique())
onetwenty.head(2)

Ireland
['NSW' 'VIC' 'WA' 'QLD' 'ACT' 'SA' 'NA' 'TAS' 'NT']


,title,link,salary,location,company,classification,summary,state
0,Shift Technician - Electrical (Data Centre),/job/37326042?type=promoted,NaN,Sydney,BGIS,Trades & Services,"BGIS, a Brookfield Company, is a leading provi...",NSW
1,Enterprise Data & Reporting Lead,/job/37393509?type=promoted,Base + Super + Profit Share,Melbourne,SEEK Limited,Information & Communication Technology,About SEEK SEEK is the global leader in the cr...,VIC


In [32]:
hundred[hundred['state'] == 'NA']
onetwenty[onetwenty['state'] == 'NA']

,title,link,salary,location,company,classification,summary,state
89,Design Engineer,/job/37426942?type=promoted,NaN,Ireland,PM Group,Engineering,We are currently looking to recruit an experie...,NA


In [33]:
# drop the one in the one hundred as it is the same as the one in one twenty
dropindex = hundred[hundred['state'] == 'NA'].index.tolist()
hundred.drop(index=dropindex, axis='index', inplace=True)

In [34]:
onefifty['state'] = onefifty['location'].apply(get_location_states)
print(onefifty['state'].unique())
onefifty.head(2)

['VIC' 'NSW' 'QLD' 'SA' 'WA' 'ACT' 'NT' 'TAS']


,title,link,salary,location,company,classification,summary,state
0,Enterprise Data & Reporting Lead,/job/37393509?type=promoted,Base + Super + Profit Share,Melbourne,SEEK Limited,Information & Communication Technology,About SEEK SEEK is the global leader in the cr...,VIC
1,C# Software Engineer |GUI | Market Data |Equit...,/job/37394390?type=promoted,NaN,Sydney,Kapital Consulting,Information & Communication Technology,About the Role An opportunity is available for...,NSW


In [35]:
twohundred['state'] = twohundred['location'].apply(get_location_states)
print(twohundred['state'].unique())
twohundred.head(2)

['VIC' 'NSW' 'QLD' 'ACT' 'SA' 'WA' 'NT']


,title,link,salary,location,company,classification,summary,state
0,Enterprise Data & Reporting Lead,/job/37393509?type=promoted,Base + Super + Profit Share,Melbourne,SEEK Limited,Information & Communication Technology,About SEEK SEEK is the global leader in the cr...,VIC
1,C# Software Engineer |GUI | Market Data |Equit...,/job/37394390?type=promoted,NaN,Sydney,Kapital Consulting,Information & Communication Technology,About the Role An opportunity is available for...,NSW


In [36]:
twofifty['state'] = twofifty['location'].apply(get_location_states)
print(twofifty['state'].unique())
twofifty.head(2)

['QLD' 'VIC' 'ACT' 'NSW' 'WA' 'SA' 'NT']


,title,link,salary,location,company,classification,summary,state
0,Data Analyst,/job/37560918?type=standout,NaN,Brisbane,Talent – Winner ‘Seek Large Recruitment Agency...,Information & Communication Technology,We are currently looking for a Data/Informatio...,QLD
1,BI Data Architect,/job/37560710?type=standout,NaN,Melbourne,KIT Consulting Group,Information & Communication Technology,Deliver key strategic change within the compl...,VIC


In [37]:
# There are still probably that some companies are duplicates with each other when combined all groups togther
# This will be dealt with later

###  Data Manipulation and Cleaning
    3. Modify salary, to add 2 columns
    
    Steps:
    * 1 column to put in the salary bracket
    * 2 column to put in the 100K or less (0) or 100K or above (1)


| Pay Group | From ('000) | To ('000)|
| --- | --- | --- |
| Sixty | 0 | 60 |
| Eight | 60 | 80 |
| Hundred | 80 | 100 |
| One Hundred Twenty | 100 | 120 |
| One Hundred Fifty | 120 | 150 |
| Two Hundred | 150 | 200 |
| Two Hundred Fifty | 200 | 250+ |


   

In [38]:
sixty['salary_bracket'] = 60  #
sixty['salary_high'] = 0 #less than 100K : 0, more than 100K : 1

In [39]:
eighty['salary_bracket'] = 80  #
eighty['salary_high'] = 0 #less than 100K : 0, more than 100K : 1

In [40]:
hundred['salary_bracket'] = 100  #
hundred['salary_high'] = 0 #less than 100K : 0, more than 100K : 1

In [41]:
onetwenty['salary_bracket'] = 120  #
onetwenty['salary_high'] = 1 #less than 100K : 0, more than 100K : 1

In [42]:
onefifty['salary_bracket'] = 150  #
onefifty['salary_high'] = 1 #less than 120K : 0, more than 120K : 1

In [43]:
twohundred['salary_bracket'] = 200  #
twohundred['salary_high'] = 1 #less than 120K : 0, more than 120K : 1

In [44]:
twofifty['salary_bracket'] = 250  #
twofifty['salary_high'] = 1 #less than 120K : 0, more than 120K : 1

In [45]:
sixty.to_csv('./datasets/cleanSixty.csv')
eighty.to_csv('./datasets/cleanEighty.csv')
hundred.to_csv('./datasets/cleanHundred.csv')
onetwenty.to_csv('./datasets/cleanOnetwenty.csv')
onefifty.to_csv('./datasets/cleanOnefifty.csv')
twohundred.to_csv('./datasets/cleanTwohundred.csv')
twofifty.to_csv('./datasets/cleanTwofifty.csv')

### Combine all of them into 1 data frame and further clean duplicates based on Summary

In [46]:
df = pd.concat([sixty, eighty, hundred, onetwenty, onefifty, twohundred, twofifty])

In [47]:
df.shape

(20156, 10)

In [48]:
# drop all duplicates except first one and show shapes
df.drop_duplicates(subset=['summary'], inplace=True)
df.shape

(14427, 10)

In [49]:
df.head()

,title,link,salary,location,company,classification,summary,state,salary_bracket,salary_high
0,Category Data Assistant,/job/37426653?type=promoted,NaN,Melbourne,TerryWhite Chemmart,Administration & Office Support,About Us TerryWhite Chemmart (TWCM) is Austral...,VIC,60,0
1,Data Administrator,/job/37381361?type=promoted,NaN,Sydney,Krunchbox Operations,Information & Communication Technology,About the business and the role This could be...,NSW,60,0
2,Data Entry,/job/37561354?type=standard,NaN,Perth,Page Personnel Office Support,Administration & Office Support,Provide Data Entry support for a 4 week period...,WA,60,0
3,Accounts Assistant / Data Entry,/job/37560786?type=standard,NaN,Perth,Perth Rad Clinic Limited,Administration & Office Support,About Perth Radiological Clinic Perth Radiolog...,WA,60,0
4,Admin /Data Entry,/job/37560693?type=standard,Up to $27.23 p.h. + Super,"Wollongong, Illawarra & South Coast",Veritas Recruitment,Administration & Office Support,The Company This market leading company requi...,NSW,60,0


### Drop all the columns that no longer in our interest

In [50]:
df.drop(columns=['link', 'salary', 'location', 'company'], inplace=True)

## Clean the Title and Summary

df['title'] = df['title'].str.replace("[-({':/})]",' ')
df['title'] = df['title'].str.replace("\\r\\n",' ')
df['title'] = df['title'].str.replace('^\w\s\&\<\>',' ')
df['title'] = df['title'].str.replace('|',' ')
df['title'] = df['title'].str.replace('–','')
df['summary'] = df['summary'].str.replace(',','')
df['title'].head()

df['summary'] = df['summary'].str.replace("[-({':/})]",' ')
df['summary'] = df['summary'].str.replace("\\r\\n",' ')
df['summary'] = df['summary'].str.replace('^\w\s\&\<\>',' ')
df['summary'] = df['summary'].str.replace(',','')
df['summary'] = df['summary'].str.replace('|','')

df['summary'].head(10)

In [51]:
df.tail()

,title,classification,summary,state,salary_bracket,salary_high
1183,Instructional Designer,Information & Communication Technology,About the companyA leading banking organisatio...,NSW,250,1
1187,Solutions Architect,Information & Communication Technology,\r\n Children’s Health Queensland is seeking t...,QLD,250,1
1190,Mainframe Analyst Programmer,Information & Communication Technology,Mainframe Analyst Programmer required for lead...,NSW,250,1
1192,Instructional Designer - Digital Literacy Trai...,Human Resources & Recruitment,Bluefin are working with a Federal government ...,NSW,250,1
1198,Frontend Team Lead,Information & Communication Technology,We are on the lookout for an experienced and p...,NSW,250,1


In [52]:
# now we have 14K rows and 6 columns
df.shape

(14427, 6)

In [53]:
df.isnull().sum()

title             0
classification    0
summary           1
state             0
salary_bracket    0
salary_high       0
dtype: int64

In [54]:
df.to_csv('./datasets/salary.csv')

In [55]:
df.sample(10)

,title,classification,summary,state,salary_bracket,salary_high
2421,Head of Marketing,Marketing & Communications,My client: A global leader within Software Con...,NSW,200,1
2013,Senior Data Engineer / Data Warehouse / Redshi...,Consulting & Strategy,About us EY DnA is the data and advanced analy...,VIC,150,1
3405,.NET Developer,Information & Communication Technology,.NET DEVELOPER - FRONTEND WIZ!!Our client is a...,NSW,120,1
1034,Senior Business Analyst,"Mining, Resources & Energy",Bengalla Mining Company is a joint venture wit...,NSW,120,1
2987,Customer Service Administration,Call Centre & Customer Service,Are you a Customer Service Star? Then look no...,NSW,80,0
1007,Government Services Program Manager,Education & Training,"Full time, fixed term until 30 June 2020 Navit...",NSW,150,1
2481,Goods Receiving - South Melbourne,"Manufacturing, Transport & Logistics",What’s in this for you? Opportunity to merge...,VIC,60,0
3776,Test Lead - ERP Implementation,Information & Communication Technology,My client is a global company who are looking ...,NSW,120,1
2035,Intermediate Accountant,Accounting,We are looking for a stand-out candidate to le...,NSW,80,0
1341,Business Analyst,Information & Communication Technology,9 months + extensions Valley location This is...,QLD,200,1
